In [64]:
import numpy as np
import pandas as pd
import inspect
import os
np.random.seed(0)

#Activation functions

def sin_function(x):
    return np.sin(x)

def sin_function_derivative(x):
    return np.cos(x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x <= 0, 0, 1)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

#Function to approximate
def f(x):
    return np.sin(3*np.pi*x + 3*np.pi/20) * np.cos(2*np.pi*x + np.pi/10) + 2

In [65]:
#Function to build the mass matrix - M
from scipy.integrate import quad

def mass_matrix(func,sigma, weights, biases, n):
    M = np.zeros([n, n])
    
    for i in range(0, n):
        for j in range(0, n):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i+1,j+1,sigma), limit=1000)[0]

    return M

In [66]:
#Single basis function - Neural Network
def single_neural_net(x,weights,biases,i, sigma): 

    return sigma(weights[i]*x+biases[i])


In [67]:
#Basis function multiplication helper for quadrature
def double_neural_net(x,weights,biases,i,j, sigma): 

    return single_neural_net(x,weights,biases,i,sigma)*single_neural_net(x,weights,biases,j,sigma)

In [68]:
#Bias helper function for quadrature
def b_neural_net(x, weights, biases,i, sigma,f):
    return single_neural_net(x,weights,biases,i, sigma)*f(x)


In [69]:
#quadrature for the bias matrix - B
def b_matrix(func,sigma, weights, biases,f, neurons):
    b = np.zeros([neurons, 1])
    for i in range(0, neurons):
            b[i] = quad(func, 0, 1, args=(weights, biases,i,sigma,f), limit=1000)[0]

    return b

In [70]:
#Function to build the approximation vector
def u_nn(x, weights, biases, sigma,collocation, neurons, a):
    u = np.zeros(collocation)
    
    for j in range(0,collocation):
        for i in range(0,neurons):
            
            u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]
        
    return u


In [71]:
#helper function for eigenvalues and eigenvectors
from scipy.linalg import eig
def get_eig(M):
    D,V = eig(M, left=False, right=True)


    idx = D.argsort()[::-1]   
    D = D[idx]
    V = V[:,idx]

    D = np.flip(D)
    V = np.fliplr(V)

    return D,V

In [72]:
#Function to calculate L2 loss
def l2_loss(f_x, u_nn_result):
    return np.linalg.norm(f_x - u_nn_result)**2

In [73]:
#Function to build the Phi matrix, made up of n basis functions
def phi_matrix(x,weights,biases,sigma,neurons,collocation):
    phi = np.zeros((collocation,neurons))
    for i in range(0,neurons):
        for j in range(0,collocation):

            phi[j,i] = sigma(weights[i]*x[j]+biases[i])
    return phi


In [74]:
def evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=False):
    phi = phi_matrix(x,weights,biases,sigma,neurons,collocation)
    f_func = f(x)
    a = np.linalg.pinv(phi)@f_func

    u = u_nn(x, weights, biases, sigma, collocation,neurons, a)

    loss = l2_loss(f_func, u)
    if verbose:
        print("="*20)
        print("Sigma:", sigma.__name__)
        print("Number of collocation points:", collocation)
        print("Number of neurons:", neurons)
        print("L2 Loss:", loss)
        print("="*20)
    return loss

In [75]:
def export_list(data, list_name, sheet_name, directory=None, columns=None, overwrite=False):
    default_directory = '/mnt/c/Git_Repos/Function_approximation/Results'
    file_name = f"{list_name}.xlsx"
    
    if columns is None:
        columns = ["Loss", "# of Collocation", "# of Neurons", "Activation Function", "R - Sample Range"]
    
    if directory is None:
        directory = default_directory
    
    full_path = os.path.join(directory, file_name)
    df = pd.DataFrame(data, columns=columns)

    if os.path.exists(full_path):
        try:
            # If the file exists
            with pd.ExcelWriter(full_path, engine='openpyxl', mode='a') as writer:
                if overwrite:
                    # If overwrite is True, delete the sheet if it exists
                    if sheet_name in writer.book.sheetnames:
                        writer.book.remove(writer.book[sheet_name])
                # Write the DataFrame to the Excel file
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        except ValueError as e:
            # If the sheet already exists and overwrite is False, print a message and continue
            print(f"Sheet name '{sheet_name}' already exists in the file '{file_name}'")
    else:
        # If the file does not exist, create a new one
        df.to_excel(full_path, index=False, sheet_name=sheet_name)

    print(f"Data exported to {full_path}")

In [76]:
def compute_loss_values(neurons, collocations, sigmas, Rs, sheet_name, xmin=0, xmax=1, overwrite=False, verbose=False):
    loss_values = []
    #np.random.seed(0)  # Set a seed for reproducibility
    for sigma in sigmas:
        for neuron in neurons:
            for collocation in collocations:
                for R in Rs:
                    x = np.linspace(xmin, xmax, collocation)
                    weights = np.random.uniform(-R, R, neuron)
                    biases = np.random.uniform(-R, R, neuron)
                    loss = evaluate_func_approx(x, f, weights, biases, sigma, neuron, collocation, verbose=verbose)
                    loss_values.append((loss, collocation, neuron, sigma.__name__,R))  # Add activation function name
    export_list(loss_values, "Finite_Neuron_Method_Results", sheet_name, overwrite=overwrite)
    return loss_values

In [77]:
def print_loss_by_collocation(loss_values):
    """
    Prints the loss values grouped by collocation point, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each collocation point
    collocation_loss_data = {collocation: [] for _, collocation, _, _, _ in loss_values}

    # Populate the dictionary with (neuron, loss, activation_func_name, R) tuples for each collocation point
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        collocation_loss_data[collocation].append((neuron, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the collocation points and print the relevant data
        for collocation, data in collocation_loss_data.items():
            print(f"# of Collocation points: {collocation}")
            
            # Print the neurons, loss values, and R for the current activation function
            for neuron, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Neurons: {neuron}, Loss: {loss:.10e}, R: {R}")



In [78]:
def print_loss_by_neuron(loss_values):
    """
    Prints the loss values grouped by neuron, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each neuron
    neuron_loss_data = {neuron: [] for _, _, neuron, _, _ in loss_values}

    # Populate the dictionary with (collocation, loss, activation_func_name, R) tuples for each neuron
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        neuron_loss_data[neuron].append((collocation, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the neurons and print the relevant data
        for neuron, data in neuron_loss_data.items():
            print(f"# of Neurons: {neuron}")
            
            # Print the collocation points, loss values, and R for the current activation function
            for collocation, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, Loss: {loss:.10e}, R: {R}")




In [79]:
def print_loss_by_R(loss_values):
    """
    Prints the loss values grouped by the range R, activation function, collocation points, and neurons.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each R value
    R_loss_data = {R: [] for _, _, _, _, R in loss_values}

    # Populate the dictionary with (collocation, neuron, loss, activation_func_name) tuples for each R value
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        R_loss_data[R].append((collocation, neuron, loss, activation_func_name))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the R values and print the relevant data
        for R, data in R_loss_data.items():
            print(f"Range R: {R}")
            
            # Print the collocation points, neurons, and loss values for the current activation function
            for collocation, neuron, loss, act_func_name in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, # of Neurons: {neuron}, Loss: {loss:.10e}")

In [80]:
collocation = 100
neurons = 50
sigma = sin_function
x = np.linspace(0,1,collocation)
weights = np.random.uniform(-1,1,neurons)
biases = np.random.uniform(-1,1,neurons)
test = evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=True)

Sigma: sin_function
Number of collocation points: 100
Number of neurons: 50
L2 Loss: 0.08512083411934636


# Test 1 - Increasing #of neurons
Keeping collocations fixed at 50, and using ReLU, tanh, and sin as activation functions

In [100]:
neurons = [5,10,20,30,40,50,100]
sigmas = [relu_derivative,tanh_derivative,sin_function_derivative]
collocations = [50]
Rs = [1]
loss_values = compute_loss_values(neurons, collocations, sigmas, Rs, "Increasing # of Neurons", overwrite=True, verbose=False)
#print_loss_by_R(loss_values_increasing_neurons)
#loss_values

Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx


/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


# Test 2 - Increasing Collocation points
Keeping neurons fixed at 20, and the same activation functions

In [101]:
neurons = [20]
sigmas = [relu_derivative,tanh_derivative,sin_function_derivative]
collocations = [10,50,100,200,500]
Rs = [1]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Increasing # of Collocation Points",overwrite=True,verbose=False)

Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx


/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [119]:
import pandas as pd

# Convert list to DataFrame
df = pd.DataFrame(loss_values, columns=['Loss', 'Collocation', 'Neurons', 'Activation Function', 'R'])

# Sort DataFrame by 'Loss' column
sorted_df = df.sort_values(by='Loss')

print("Sorted by Loss")
print(sorted_df)
print("="*80)
print("")
print("Filtered for Sin Activation Function")
filtered_df = df[df["Activation Function"] == "sin_function"]

print(filtered_df)

Sorted by Loss
        Loss  Collocation  Neurons Activation Function   R
13  0.000002  50           30       sin_function        5 
1   0.000017  50           30       tanh                2 
3   0.000037  50           30       tanh                5 
0   0.000128  50           30       tanh                1 
14  0.000202  50           30       sin_function        10
4   0.000257  50           30       tanh                10
12  0.000656  50           30       sin_function        4 
11  0.000814  50           30       sin_function        2 
2   0.001043  50           30       tanh                4 
10  0.045561  50           30       sin_function        1 
5   0.858394  50           30       relu                1 
8   1.508541  50           30       relu                5 
9   2.265392  50           30       relu                10
7   2.906647  50           30       relu                4 
6   3.156628  50           30       relu                2 

Filtered for Sin Activation Function
   

In [83]:
#print_loss_by_neuron(loss_values)

# Test 3 - Increasing the sampling range R
Keeping neurons and collocations fixed, increasing R to increase chance that w_i and b_i are further away from w_j and b_j

In [102]:
neurons = [30]
sigmas = [tanh,relu,sin_function]
sigmas_derivative = [tanh_derivative,relu_derivative,sin_function_derivative]
collocations = [50]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Increasing R",verbose=False)
derivative_loss_values = compute_loss_values(neurons, collocations,sigmas_derivative,Rs,"Increasing R - Derivative activations",verbose=False)

/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Sheet name 'Increasing R' already exists in the file 'Finite_Neuron_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx
Sheet name 'Increasing R - Derivative activations' already exists in the file 'Finite_Neuron_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx


In [105]:
comparison_values = loss_values + derivative_loss_values
df = pd.DataFrame(comparison_values, columns=["Loss", "# of Collocation", "# of Neurons", "Activation Function", "R - Sample Range"])

# Export the DataFrame to an Excel file
df.to_excel("combined_loss_values.xlsx", index=False)

print("Data exported to combined_loss_values.xlsx")

Data exported to combined_loss_values.xlsx


In [106]:
df

,Loss,# of Collocation,# of Neurons,Activation Function,R - Sample Range
0,1.278497e-04,50,30,tanh,1
1,1.692095e-05,50,30,tanh,2
2,1.043494e-03,50,30,tanh,4
3,3.653896e-05,50,30,tanh,5
4,2.567580e-04,50,30,tanh,10
5,8.583941e-01,50,30,relu,1
6,3.156628e+00,50,30,relu,2
7,2.906647e+00,50,30,relu,4
8,1.508541e+00,50,30,relu,5
9,2.265392e+00,50,30,relu,10


In [85]:
#print_loss_by_collocation(derivative_loss_values)

In [86]:
function_dict = {(func, r): err for err, _, _, func, r in loss_values}
derivative_dict = {(func, r): err for err, _, _, func, r in derivative_loss_values}

def base_function_name(derivative_name):
    if derivative_name.endswith('_derivative'):
        return derivative_name.replace('_derivative', '')
    return None

comparison_results = []
for (func_name, r), err in derivative_dict.items():
    base_name = base_function_name(func_name)
    if base_name:
        base_err = function_dict.get((base_name, r))
        if base_err is not None:
            comparison_results.append((base_name, r, base_err, err))

print(f"{'Function':<20}{'R Value':<10}{'Function Error':<20}{'Derivative Error':<25}")
print("="*80)
for base_name, r, base_err, der_err in comparison_results:
    print(f"{base_name:<20}{r:<10}{base_err:<20.10e}{der_err:<25.10e}")


Function            R Value   Function Error      Derivative Error         
tanh                1         2.3434375050e-05    3.2283148589e-06         
tanh                2         3.5857798345e-05    6.3889155757e-04         
tanh                4         2.3862848033e-09    1.4912348109e-06         
tanh                5         7.0521932236e-04    3.8311405099e-08         
tanh                10        1.1318563900e-03    4.0555510274e-10         
relu                1         1.1869997774e+00    2.5778510505e+00         
relu                2         2.8164332146e+00    3.6305058863e+00         
relu                4         4.5088958398e+00    2.7595219648e+00         
relu                5         2.9962341889e+00    3.7695511148e+00         
relu                10        4.0318911069e-01    1.7786649808e+00         
sin_function        1         4.5290035084e-02    4.3091522541e-02         
sin_function        2         8.5666752506e-04    8.9595396603e-04         
sin_function

In [87]:
neurons = [10,20,30,50,100]
sigmas = [tanh,relu,sin_function]
collocations = [5,10,30,50,100]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Full_tests",verbose=False)

Sheet name 'Full_tests' already exists in the file 'Finite_Neuron_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx


/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [88]:
#print_loss_by_collocation(loss_values)

# Building Mass matrix

In [89]:
from scipy.integrate import quad

def mass_matrix(func,sigma, weights, biases, neurons):
    M = np.zeros([neurons, neurons])
    
    for i in range(0, neurons):
        for j in range(0, neurons):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i,j,sigma), limit=1000)[0]

    return M

In [90]:
neurons = 10
R = 10
weights = np.random.uniform(-R,R,neurons)
biases = np.random.uniform(-R,R,neurons)
sigma = tanh
M = mass_matrix(double_neural_net,sigma, weights, biases, neurons)

In [91]:
#M

In [92]:
det_m = np.linalg.det(M)
#det_m

In [93]:
b = b_matrix(b_neural_net,sigma,weights, biases,f,neurons)
#b

In [94]:
a = np.linalg.solve(M,b)
#a

In [95]:
collocation = 100
x = np.linspace(0,1,collocation)
u = u_nn(x,weights,biases,sin_function,collocation,neurons,a)

/tmp/ipykernel_21123/1981966750.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]


In [96]:
loss = l2_loss(f(x), u)
print("L2 Loss:", loss)

L2 Loss: 41210325357564.1
